In [1]:
%load_ext autoreload
%autoreload 2

import torch

from torch.utils.data import DataLoader
from dataset import RecoveryRateDataset
import os
from training_utils import compute_mean_and_std, apply_std_scaling, set_seed
from model_lightning import LSTMEstimator
from training_loop import model_train

from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.callbacks import ModelCheckpoint, LearningRateMonitor
from pytorch_lightning.profiler import PyTorchProfiler
from pytorch_lightning.loggers import WandbLogger

SEED = 1234
seed_everything(SEED, workers=True)

import os
os.environ["WANDB_START_METHOD"] = "thread"
import wandb

/home/cal-display1/Documents/4tb/aksoy/start_env/lib/python3.9/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
Global seed set to 1234


In [2]:
root_dir = os.getcwd().split('Start_PyCharm')[0]
data_file_path = root_dir + 'Start_PyCharm/state_vector_dataframes_as_training_data/data/airport_state_df_w_features.pickle'


SEQUENCE_LENGTH = 3

features_to_drop = ['reg_type', 'reg_bool_type', 'reg_cause']
rr_dataset = RecoveryRateDataset(data_file_path, features_to_drop=features_to_drop, sequence_length=SEQUENCE_LENGTH,
                                 fill_with='backfill')


train_val_test_ratios = (0.6, 0.2, 0.2)

lengths=[int(len(rr_dataset)*ratio) for ratio in train_val_test_ratios]

if sum(lengths) != len(rr_dataset):
    offset = sum(lengths) - len(rr_dataset)
    lengths[0] = lengths[0] - offset
                                                                      
train, val, test = torch.utils.data.random_split(rr_dataset, lengths=lengths)

#Compute mean and std for standard scaling.
#training_mean, training_std = compute_mean_and_std(train)

#scaled_training_data = apply_std_scaling(train, training_mean, training_std)
#scaled_val_data = apply_std_scaling(val, training_mean, training_std)
#scaled_test_data = apply_std_scaling(test, training_mean, training_std)

In [3]:
val_data_loader = DataLoader(val, batch_size=64, num_workers=4)
test_data_loader = DataLoader(test)

In [5]:
experiment = [16, 32, 64, 128, 256]

for parameter in experiment:
    BATCH_SIZE = parameter
    
    training_data_loader = DataLoader(train, batch_size=BATCH_SIZE, shuffle=True,
                                  num_workers=4)
    main(parameter)

wandb: Currently logged in as: aksoym (use `wandb login --relogin` to force relogin)


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                      | Type       | Params
---------------------------------------------------------
0 | feature_extracting_layers | Sequential | 2.5 K 
1 | lstm_layer                | LSTM       | 16.9 K
2 | estimator_layers          | Sequential | 3.1 K 
---------------------------------------------------------
22.5 K    Trainable params
0         Non-trainable params
22.5 K    Total params
0.090     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 1234


Training: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Time limit reached. Elapsed time is 2:00:00. Signaling Trainer to stop.


Validating: 0it [00:00, ?it/s]

FIT Profiler Report
Profile stats for: records
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                          ProfilerStep*        13.18%       2.917ms        99.76%      22.083ms       7.361ms       0.000us         0.00%     294.000us      98.000us             3  
enumerate(DataLoader)#_MultiProcessingDataLoaderIter...        56.29%      12.461ms        56.66%      12.543ms       4.181ms       0.000us         0.00%       

In [4]:
def main(batch_size):
    #Training parameters.
    LR = 0.0001

    model = LSTMEstimator(len(rr_dataset.feature_names), initial_dense_layer_size=32, dense_parameter_multiplier=1,
                          dense_layer_count=2, lstm_layer_count=2, lstm_hidden_units=32, dropout=0.4,
                          sequence_length=SEQUENCE_LENGTH, lr=LR)


    #wandb_instance = wandb.init(project='recovery_rate')  
    profiler = PyTorchProfiler()
    lr_monitor = LearningRateMonitor(logging_interval="epoch")
    wandb_logger = WandbLogger(name=f"batch_s={batch_size}", project='recovery_rate', log_model="all")
    checkpoint_callback = ModelCheckpoint(monitor="val_loss", mode="min")
    wandb_logger.experiment.config.update({'batch_size': batch_size, 'init_lr': LR})
    

    lightning_trainer = Trainer(logger=wandb_logger, devices="auto", callbacks=[checkpoint_callback, lr_monitor], 
                                deterministic=True, accelerator="gpu", max_time="00:02:00:00", profiler=profiler)
    
    
    lightning_trainer.fit(model, training_data_loader, val_data_loader)
    wandb_logger.finalize("completed")
    wandb.finish(quiet=True)
    